# imports and setup

In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install "xformers<0.0.26" trl peft accelerate bitsandbytes pinecone-client sentence-transformers langchain-openai openai

In [ ]:
import torch
from datasets import load_dataset
from huggingface_hub import notebook_login
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import numpy as np

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 256,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 512,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

tokenizer.padding_side = "left"

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key='INSERT_PINECONE_API_KEY_HERE')

In [ ]:
index = pc.Index("medmcqa-train-1")

In [ ]:
from langchain_openai import OpenAIEmbeddings
embed_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key="INSERT_OPEN_API_KEY_HERE")

def get_relevant_context(text, k=1):
  embeds = embed_model.embed_documents([text])
  results_1 = index.query(
      vector=embeds[0],
      top_k=k,
      include_metadata=True
  )

  results = []

  for result in results_1["matches"]:
    results.append(result["metadata"]["text"])
  return results

# load datasets

In [ ]:
medmcqa = load_dataset("openlifescienceai/medmcqa", split="validation")
medmcqa_mixed = medmcqa.select(range(234))
medmcqa_anatomy = medmcqa.filter(lambda example: example["subject_name"] == "Anatomy")

medqa = load_dataset("GBaker/MedQA-USMLE-4-options", split="test").select(range(234))

pubmedqa = load_dataset("bigbio/pubmed_qa", split="validation").select(range(234))

mmlu_anatomy = load_dataset("openlifescienceai/mmlu_anatomy", split="test")

In [ ]:
print(medmcqa_mixed)
print(medmcqa_anatomy)
print(medqa)
print(pubmedqa)
print(mmlu_anatomy)

In [ ]:
FastLanguageModel.for_inference(model)

In [ ]:
def predict(prompt):
    inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
    return tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens = True)[0].strip()

# medmcqa mixed

In [ ]:
def eval_medmcqa_mixed():
  references = []
  predictions = []
  SAMPLE_CNT = len(medmcqa_mixed)
  mismatch_cnt = 0
  n_gram_total = [0, 0, 0, 0, 0]
  n_gram_correct = [0, 0, 0, 0, 0]

  for i in range(SAMPLE_CNT):
    example = medmcqa_mixed[i]
    options = [example['opa'], example['opb'], example['opc'], example['opd']]

    prompt = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n" + example["question"] + "\n\n" + example['opa'] + "\n" + example['opb'] + "\n" + example['opc'] + "\n" + example['opd'] + "\n\nRespond with the correct choice from the list above verbatim. Do not include any explanation."

    context = get_relevant_context(example["question"] + "\n\n" + example['opa'] + "\n" + example['opb'] + "\n" + example['opc'] + "\n" + example['opd'])
    prompt += " You may use the following information only if it is helpful: \n" + context[0]

    prompt += "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    prediction = predict(prompt)

    if prediction not in options:
      prompt += prediction + "<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n\nYour response does not exactly match one of the choices from the list. Do not apologise or include any text other than one of the options from the list verbatim without any label. Here are the options again\n\n" + example['opa'] + "\n\n" + example['opb'] + "\n\n" + example['opc'] + "\n\n" + example['opd'] + "\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
      prediction = predict(prompt)

    reference = options[example['cop']]

    for i in options:
      grams = round(sum([len(option.split()) for option in options]) / len(options))

    if grams <= 5:
      n_gram_total[grams - 1] += 1
      n_gram_correct[grams - 1] += prediction == reference

    if prediction not in options:
      mismatch_cnt += 1

    references.append(reference)
    predictions.append(prediction)

  em = sum([1 if prediction == reference else 0 for prediction, reference in zip(predictions, references)]) / SAMPLE_CNT
  mismatch = mismatch_cnt / SAMPLE_CNT
  em_by_n_gram = [correct / total for correct, total in zip(n_gram_correct, n_gram_total)]

  return em, mismatch, em_by_n_gram

em, mismatch, em_by_n_gram = eval_medmcqa_mixed()

In [ ]:
print("exact match:", em)
print("mismatch:", mismatch)
print("em by n gram:", em_by_n_gram)

exact match: 0.5854700854700855
mismatch: 0.01282051282051282
em by n gram: [0.5555555555555556, 0.6190476190476191, 0.56, 0.5, 0.5384615384615384]


# medmcqa anatomy

In [ ]:
def eval_medmcqa_anatomy():
  references = []
  predictions = []
  SAMPLE_CNT = len(medmcqa_anatomy)
  mismatch_cnt = 0
  n_gram_total = [0, 0, 0, 0, 0]
  n_gram_correct = [0, 0, 0, 0, 0]

  for i in range(SAMPLE_CNT):
    example = medmcqa_anatomy[i]
    options = [example['opa'], example['opb'], example['opc'], example['opd']]

    prompt = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n" + example["question"] + "\n\n" + example['opa'] + "\n" + example['opb'] + "\n" + example['opc'] + "\n" + example['opd'] + "\n\nRespond with the correct choice from the list above verbatim. Do not include any explanation."

    context = get_relevant_context(example["question"] + "\n\n" + example['opa'] + "\n" + example['opb'] + "\n" + example['opc'] + "\n" + example['opd'])
    prompt += " You may use the following information only if it is helpful: \n" + context[0]

    prompt += "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    prediction = predict(prompt)

    if prediction not in options:
      prompt += prediction + "<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n\nYour response does not exactly match one of the choices from the list. Do not apologise or include any text other than one of the options from the list verbatim without any label. Here are the options again\n\n" + example['opa'] + "\n\n" + example['opb'] + "\n\n" + example['opc'] + "\n\n" + example['opd'] + "\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
      prediction = predict(prompt)

    reference = options[example['cop']]

    for i in options:
      grams = round(sum([len(option.split()) for option in options]) / len(options))

    if grams <= 5:
      n_gram_total[grams - 1] += 1
      n_gram_correct[grams - 1] += prediction == reference

    if prediction not in options:
      mismatch_cnt += 1

    references.append(reference)
    predictions.append(prediction)

  em = sum([1 if prediction == reference else 0 for prediction, reference in zip(predictions, references)]) / SAMPLE_CNT
  mismatch = mismatch_cnt / SAMPLE_CNT
  em_by_n_gram = [correct / total for correct, total in zip(n_gram_correct, n_gram_total)]

  return em, mismatch, em_by_n_gram

em, mismatch, em_by_n_gram = eval_medmcqa_anatomy()

In [ ]:
print("exact match:", em)
print("mismatch:", mismatch)
print("em by n gram:", em_by_n_gram)

exact match: 0.6367521367521367
mismatch: 0.021367521367521368
em by n gram: [0.6862745098039216, 0.6213592233009708, 0.7586206896551724, 0.4583333333333333, 0.625]


# medqa

In [ ]:
def eval_medqa():
  references = []
  predictions = []
  SAMPLE_CNT = len(medqa)
  mismatch_cnt = 0
  both_right = 0
  both_wrong = 0
  only_rag = 0
  only_without = 0
  n_gram_total = [0, 0, 0, 0, 0]
  n_gram_correct = [0, 0, 0, 0, 0]

  for i in range(SAMPLE_CNT):
    example = medqa[i]
    options = [example['options']['A'], example['options']['B'], example['options']['C'], example['options']['D']]

    prompt = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n" + example["question"] + "\n\n" + example['options']['A'] + "\n" + example['options']['B'] + "\n" + example['options']['C'] + "\n" + example['options']['D'] + "\n\nRespond with the correct choice from the list above verbatim. Do not include any explanation."

    context = get_relevant_context(example["question"] + "\n\n" + example['options']['A'] + "\n" + example['options']['B'] + "\n" + example['options']['C'] + "\n" + example['options']['D'])
    prompt += " You may use the following information only if it is helpful: \n" + context[0]

    prompt += "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

    prediction = predict(prompt)

    if prediction not in options:
      prompt += prediction + "<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n\nYour response does not exactly match one of the choices from the list. Do not apologise or include any text other than one of the options from the list verbatim without any label. Here are the options again\n\n" + example['options']['A'] + "\n\n" + example['options']['B'] + "\n\n" + example['options']['C'] + "\n\n" + example['options']['D'] + "\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
      prediction = predict(prompt)

    if prediction not in options:
      mismatch_cnt += 1

    reference = example['options'][example['answer_idx']]

    for i in options:
      grams = round(sum([len(option.split()) for option in options]) / len(options))

    if grams <= 5:
      n_gram_total[grams - 1] += 1
      n_gram_correct[grams - 1] += prediction == reference

    references.append(reference)
    predictions.append(prediction)

  em = sum([1 if prediction == reference else 0 for prediction, reference in zip(predictions, references)]) / SAMPLE_CNT
  mismatch = mismatch_cnt / SAMPLE_CNT
  em_by_n_gram = [correct / total for correct, total in zip(n_gram_correct, n_gram_total)]

  return em, mismatch, em_by_n_gram

em, mismatch, em_by_n_gram = eval_medqa()

In [ ]:
print("exact match:", em)
print("mismatch:", mismatch)
print("em by n gram:", em_by_n_gram)

exact match: 0.5811965811965812
mismatch: 0.01282051282051282
em by n gram: [0.5945945945945946, 0.6341463414634146, 0.4827586206896552, 0.625, 0.5294117647058824]


# pubmedqa

In [ ]:
def eval_pubmedqa():
  references = []
  predictions = []
  SAMPLE_CNT = len(pubmedqa)
  mismatch_cnt = 0

  for i in range(SAMPLE_CNT):
    example = pubmedqa[i]

    prompt = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n" + example["QUESTION"] + "\n\nRespond with only lower case yes or lowercase no."

    context = get_relevant_context(example["QUESTION"] + "\n\n")
    prompt += " You may use the following information only if it is helpful: \n" + context[0]

    prompt += "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

    prediction = predict(prompt)

    if prediction not in ["yes", "no"]:
      prompt += prediction + "<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n\nYour response does not exactly match yes or no. Do not apologise, simply respond with yes or no\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
      prediction = predict(prompt)

    if prediction not in ["yes", "no"]:
      mismatch_cnt += 1

    reference = example['final_decision']

    references.append(reference)
    predictions.append(prediction)

    em = sum([1 if prediction == reference else 0 for prediction, reference in zip(predictions, references)]) / SAMPLE_CNT

    mismatch =  mismatch_cnt / SAMPLE_CNT

  return em, mismatch

em, mismatch = eval_pubmedqa()

In [ ]:
print("exact match:", em)
print("mismatch:", mismatch)

# mmlu anatomy

In [ ]:
def eval_mmlu_anatomy():
  references = []
  predictions = []
  SAMPLE_CNT = len(mmlu_anatomy)
  mismatch_cnt = 0
  n_gram_total = [0, 0, 0, 0, 0]
  n_gram_correct = [0, 0, 0, 0, 0]

  for i in range(SAMPLE_CNT):
    example = mmlu_anatomy[i]["data"]
    options = [example["Options"]["A"], example["Options"]["B"], example["Options"]["C"], example["Options"]["D"]]

    prompt = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n" + example["Question"] + "\n\n" + example['Options']['A'] + "\n" + example['Options']['B'] + "\n" + example['Options']['C'] + "\n" + example['Options']['D'] + "\n\nRespond with the correct choice from the list above verbatim. Do not include any explanation."

    context = get_relevant_context(example["Question"] + "\n\n" + example['Options']['A'] + "\n" + example['Options']['B'] + "\n" + example['Options']['C'] + "\n" + example['Options']['D'])
    prompt += " You may use the following information only if it is helpful: \n" + context[0]

    prompt += "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

    prediction = predict(prompt)

    if prediction not in options:
      prompt += prediction + "<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n\nYour response does not exactly match one of the choices from the list. Do not apologise or include any text other than one of the options from the list verbatim without any label. Here are the options again\n\n" + example['Options']['A'] + "\n\n" + example['Options']['B'] + "\n\n" + example['Options']['C'] + "\n\n" + example['Options']['D'] + "\n\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
      prediction = predict(prompt)

    if prediction not in options:
      mismatch_cnt += 1

    reference = example["Correct Answer"]

    for i in options:
      grams = round(sum([len(option.split()) for option in options]) / len(options))

    if grams <= 5:
      n_gram_total[grams - 1] += 1
      n_gram_correct[grams - 1] += prediction == reference

    references.append(reference)
    predictions.append(prediction)

    em = sum([1 if prediction == reference else 0 for prediction, reference in zip(predictions, references)]) / SAMPLE_CNT
    mismatch =  mismatch_cnt / SAMPLE_CNT
    em_by_n_gram = [correct / total if total != 0 else -1 for correct, total in zip(n_gram_correct, n_gram_total)]

  return em, mismatch, em_by_n_gram

em, mismatch, em_by_n_gram = eval_mmlu_anatomy()

In [ ]:
print("exact match:", em)
print("mismatch", mismatch)
print("em by n gram:", em_by_n_gram)

exact match: 0.5777777777777777
mismatch 0.05185185185185185
em by n gram: [0.9090909090909091, 0.5909090909090909, 0.3333333333333333, 0.4444444444444444, 0.4]
